In [25]:
# requirements
# Инициализация
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage, SystemMessage

# устанавливаем локаль
import locale
locale.setlocale(locale.LC_ALL, ('ru', 'utf-8'))
print(locale.getlocale())

# Загрузка документов
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter
)

# Эмбеддинги
from chromadb.config import Settings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GigaChatEmbeddings

# QnA цепочка
from langchain.chains import RetrievalQA
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)

('ru_RU', 'UTF-8')


In [26]:
import os
from dotenv import load_dotenv
load_dotenv()
token = os.getenv("token")
data_file = "./data/bzd.docx"

In [27]:
llm = GigaChat(
    credentials=token,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False)

In [28]:
messages = [
    SystemMessage(
        content="Ты эмпатичный бот-психолог, который помогает пользователю решить его проблемы."
    )
]

user_input = input("User: ")
messages.append(HumanMessage(content=user_input))
res = llm(messages)
messages.append(res)
# Ответ сервиса
print("Bot: ", res.content)

Bot:  Среди знаменитостей, родившихся в год распада СССР (1991), есть следующие:

— Гоша Куценко — российский актёр, режиссёр, сценарист и продюсер;
— Скарлетт Йоханссон — американская актриса и певица;
— Эванджелин Лилли — канадская актриса и модель;
— Криштиану Роналду — португальский футболист;
— Аманда Сейфрид — американская актриса и певица;
— Майли Сайрус — американская актриса и певица;
— Лиам Хемсворт — австралийский актёр;
— Тейлор Лотнер — американский актёр и музыкант;
— Селена Гомес — американская актриса, певица и дизайнер;
— Николас Холт — британский актёр.


In [29]:
loader = Docx2txtLoader(data_file)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 446


In [30]:
embeddings = GigaChatEmbeddings(
    one_by_one_mode=True,
    _debug_delay=0.005,
    credentials=token,
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=True),
)

retriever = db.as_retriever(k=10)

In [31]:
question = "Устойчивость работы хозяйственного объекта определяется по"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain({"query": question}))

{'query': 'Устойчивость работы хозяйственного объекта определяется по', 'result': 'Из представленного текста нельзя однозначно определить, как именно определяется устойчивость работы хозяйственного объекта.'}


In [32]:
question = "Аварией считается утечка нефти в обьеме"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain({"query": question}))

{'query': 'Аварией считается утечка нефти в обьеме', 'result': 'Аварией считается утечка нефти в объеме 10 м3 и более.'}


In [33]:
question = "Пожарная техника в зависимости от способа пожаротушения подразделяется на"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain({"query": question}))

{'query': 'Пожарная техника в зависимости от способа пожаротушения подразделяется на', 'result': 'Пожарная техника в зависимости от способа пожаротушения подразделяется на первичные средства (огнетушители и пожарные краны), передвижные средства (различные пожарные автомобили) и стационарные средства (специальные установки с запасом огнетушащих веществ, лафетные стволы и другие).'}


In [34]:
question = "Сколько классов опасных производственных объектов существует"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())
print(qa_chain({"query": question}))

{'query': 'Сколько классов опасных производственных объектов существует', 'result': 'Согласно Федеральному закону №116-ФЗ "О промышленной безопасности опасных производственных объектов", существует четыре класса опасных производственных объектов. Это классификация основывается на степени отклонения фактических уровней негативных факторов от гигиенических нормативов условий труда.'}


In [35]:
question = "Кто такой джастин бибер?"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain({ "query": question}))

{'query': 'Кто такой джастин бибер?', 'result': 'Джастин Бибер - это канадский певец, автор песен и актер. Он стал популярным в возрасте 13 лет благодаря видеороликам на YouTube, где он исполнял песни собственного сочинения. Впоследствии его обнаружил менеджер Скутер Браун и помог ему подписать контракт с Island Records в 2008 году.\n\nМузыкальная карьера Джастина Бибера началась с выпуска сингла "One Time", который достиг топ-10 в Канаде и США. После этого последовал выпуск нескольких успешных альбомов, включая "My World", "My World 2.0", "Under the Mistletoe", "Believe" и "Purpose". Его музыка сочетает в себе элементы поп-музыки, R&B и хип-хопа.\n\nПомимо музыкальной карьеры, Джастин Бибер также активно участвует в общественной жизни. Он занимается благотворительностью, поддерживая различные организации и фонды. Кроме того, он снялся в нескольких фильмах, включая "Никогда не говори никогда" (документальный фильм о его концертном туре) и "Путеводная звезда" (рождественская комедия).'}

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain
sys_templ = '''Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, скажи "Я не знаю".
<context>
{context}
</context>
'''
qa_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            sys_templ,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
combine_docs_chain = create_stuff_documents_chain(llm, qa_prompt)

from langchain_core.messages import HumanMessage

# question = "Кто такой джастин бибер?"
question = "Устойчивость работы хозяйственного объекта определяется по"
question = "Сколько классов опасных производственных объектов существует"

# Поиск по базе данных
docs = retriever.invoke(question)

result = combine_docs_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content=question)
        ],
    }
)

print(result)

Согласно тексту, опасные производственные объекты подразделяются на четыре класса.
